In [6]:
import pandas as pd
from sklearn.metrics import accuracy_score
import re
from collections import OrderedDict

In [7]:
X_train = pd.read_excel('full-data/X_train.xlsx')

In [8]:
X_test = pd.read_excel('full-data/X_test.xlsx')

In [9]:
y_train = pd.read_excel('full-data/y_train.xlsx')

In [10]:
y_test = pd.read_excel('full-data/y_test.xlsx')

In [11]:
# Baseline #1 -> Just use W-V and W-H to see who wins, if tie then give to H
baseline_preds = []
for r, row in X_test.iterrows():
    W_V = row['W-V']
    L_V = row['L-V']
    W_H = row['W-H']
    L_H = row['L-H']
    choice = 1 if W_H >= W_V else 0
    baseline_preds.append(choice)
accuracy_score(baseline_preds, y_test)

0.7021362353889561

In [12]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import GridSearchCV
kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [13]:
X_train_data = X_train.values
X_test_data = X_test.values
y_train_data = y_train.values.flatten()
y_test_data = y_test.values.flatten()

In [14]:
# XGBoost
from xgboost import XGBClassifier

xgb = XGBClassifier(n_jobs=-1)
param_grid_xgb = {
    'n_estimators': [100, 200, 500, 1000],
    'learning_rate': [.001, .01, .05, .1, 1]
}
grid_search_xgb = GridSearchCV(xgb, param_grid_xgb, cv=kf, n_jobs=-1)
grid_search_xgb.fit(X_train_data, y_train_data, early_stopping_rounds=5, eval_set=[(X_test_data, y_test_data)], verbose=False)

# xgb.fit(X_train_data, y_train_data, early_stopping_rounds=5, eval_set=[(X_test_data, y_test_data)], verbose=False)

print(grid_search_xgb.best_params_)
print(grid_search_xgb.best_score_)
# pd.DataFrame(grid_search_xgb.cv_results_)

{'learning_rate': 0.1, 'n_estimators': 100}
0.6689516129032258


In [18]:
feature_names = {num: val for num, val in enumerate(X_train.columns.tolist())}

In [19]:
feature_gain = grid_search_xgb.best_estimator_.get_booster().get_score(importance_type='gain')

In [28]:
# Make a dataframe for the feature importances
feature_importances = []
for i, name in enumerate(feature_names.values()):
    label = 'f{}'.format(i)
    gain = gain = feature_gain[label] if label in feature_gain else None
    feature_importances.append([name, gain])
importance_df = pd.DataFrame(feature_importances, columns=['Feature', 'Gain'])

In [32]:
importance_df.sort_values(by='Gain', ascending=False)

,Feature,Gain
240,NetRtg-H,57.761836
249,PIE-V,45.577457
241,NetRtg-V,32.989168
248,PIE-H,24.732565
220,WS/48-S1-H,22.123787
...,...,...
250,REB%-H,NaN
251,REB%-V,NaN
253,TOV%-V,NaN
254,TS%-H,NaN


In [33]:
importance_df.to_excel('feature_importance.xlsx', index=False)

C:\Users\Max Brenner\Anaconda3\lib\site-packages\pandas\compat\_optional.py:106: UserWarning: Pandas requires version '0.9.8' or newer of 'xlsxwriter' (version '0.9.6' currently installed).
  warnings.warn(msg, UserWarning)


In [34]:
cross_val_score(grid_search_xgb.best_estimator_, X_test_data, y_test_data, cv=kf).mean()

C:\Users\Max Brenner\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Max Brenner\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Max Brenner\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Max Brenner\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous

0.6767459596287402

In [37]:
y_pred = grid_search_xgb.best_estimator_.predict(X_test_data)
accuracy_score(y_test, y_pred)

C:\Users\Max Brenner\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.6968964127367997

In [38]:
# See how much the preds differ from baseline
accuracy_score(baseline_preds, y_pred)

0.8182184602982668

In [39]:
# From above it seems that the model is more similar to the baseline (just using wins to answer) then the target

In [35]:
X_train_pre_drop = pd.read_excel('full-data/X_train_pre_drop.xlsx')
X_test_pre_drop = pd.read_excel('full-data/X_test_pre_drop.xlsx')

In [40]:
# Get when the model was wrong
incorrects = []
for t, p in zip(y_test.values.flatten(), y_pred):
    if t != p: incorrects.append(1)
    else: incorrects.append(0)

In [41]:
# Get when the baseline was wrong
incorrects_baseline = []
for t, p in zip(y_test.values.flatten(), baseline_preds):
    if t != p: incorrects_baseline.append(1)
    else: incorrects_baseline.append(0)

In [42]:
X_test_pre_drop.head(2)

,Date,Visitor,PTS-V,Home,PTS-H,W-V,L-V,W-H,L-H,Rest-V,...,PIE-H,PIE-V,REB%-H,REB%-V,TOV%-H,TOV%-V,TS%-H,TS%-V,eFG%-H,eFG%-V
0,Sun Nov 24 2019,Sacramento Kings,113,Washington Wizards,106,7,8,5,9,2,...,47.8,47.0,47.8,48.9,13.4,14.8,56.6,55.8,52.9,52.7
1,Sat Jan 9 2016,Chicago Bulls,105,Atlanta Hawks,120,22,13,23,15,2,...,52.8,49.4,47.4,50.6,15.2,14.2,55.2,52.6,51.6,48.7


In [96]:
# Tally when in the season games too place that the model predicted incorrectly
# And tally years
months_ticks = OrderedDict({'Oct': 0, 'Nov': 0, 'Dec': 0, 'Jan': 0, 'Feb': 0, 'Mar': 0, 'Apr': 0})
months_totals = OrderedDict({'Oct': 0, 'Nov': 0, 'Dec': 0, 'Jan': 0, 'Feb': 0, 'Mar': 0, 'Apr': 0})

years_ticks = OrderedDict({2015: 0, 2016: 0, 2017: 0, 2018: 0, 2019: 0})
years_totals = OrderedDict({2015: 0, 2016: 0, 2017: 0, 2018: 0, 2019: 0})

def which_season(year, month):
    if month in ['Oct', 'Nov', 'Dec']:
        return int(year)
    else:
        return int(year) - 1

for (r, row), inc in zip(X_test_pre_drop.iterrows(), incorrects):
    date_regex = re.match(r'\w+\s(\w+)\s(\d+)\s(\d+)', row['Date'])
    month = date_regex.group(1)
    year = date_regex.group(3)
    which_seas = which_season(year, month)
    years_totals[which_seas] += 1
    months_totals[month] += 1
    if not inc:
        continue
    months_ticks[month] += 1
    years_ticks[which_seas] += 1

for (month, tick), total in zip(list(months_ticks.items()), list(months_totals.values())):
    print('{}: {}/{} ticks  {} proportion'.format(month, tick, total, tick/total if not total==0 else 'N/A'))
print('\n\n')
for (year, tick), total in zip(list(years_ticks.items()), list(years_totals.values())):
    print('{}: {}/{} ticks  {} proportion'.format(year, tick, total, tick/total if not total==0 else 'N/A'))

Oct: 0/0 ticks  N/A proportion
Nov: 103/358 ticks  0.2877094972067039 proportion
Dec: 171/555 ticks  0.3081081081081081 proportion
Jan: 180/568 ticks  0.31690140845070425 proportion
Feb: 117/381 ticks  0.30708661417322836 proportion
Mar: 123/431 ticks  0.2853828306264501 proportion
Apr: 58/188 ticks  0.30851063829787234 proportion



2015: 149/548 ticks  0.2718978102189781 proportion
2016: 187/548 ticks  0.34124087591240876 proportion
2017: 170/530 ticks  0.32075471698113206 proportion
2018: 150/528 ticks  0.2840909090909091 proportion
2019: 96/327 ticks  0.29357798165137616 proportion


In [97]:
# BASELINE

# Tally when in the season games too place that the model predicted incorrectly
# And tally years
months_ticks = OrderedDict({'Oct': 0, 'Nov': 0, 'Dec': 0, 'Jan': 0, 'Feb': 0, 'Mar': 0, 'Apr': 0})
months_totals = OrderedDict({'Oct': 0, 'Nov': 0, 'Dec': 0, 'Jan': 0, 'Feb': 0, 'Mar': 0, 'Apr': 0})

years_ticks = OrderedDict({2015: 0, 2016: 0, 2017: 0, 2018: 0, 2019: 0})
years_totals = OrderedDict({2015: 0, 2016: 0, 2017: 0, 2018: 0, 2019: 0})

def which_season(year, month):
    if month in ['Oct', 'Nov', 'Dec']:
        return int(year)
    else:
        return int(year) - 1

for (r, row), inc in zip(X_test_pre_drop.iterrows(), incorrects_baseline):
    date_regex = re.match(r'\w+\s(\w+)\s(\d+)\s(\d+)', row['Date'])
    month = date_regex.group(1)
    year = date_regex.group(3)
    which_seas = which_season(year, month)
    years_totals[which_seas] += 1
    months_totals[month] += 1
    if not inc:
        continue
    months_ticks[month] += 1
    years_ticks[which_seas] += 1

for (month, tick), total in zip(list(months_ticks.items()), list(months_totals.values())):
    print('{}: {}/{} ticks  {} proportion'.format(month, tick, total, tick/total if not total==0 else 'N/A'))
print('\n\n')
for (year, tick), total in zip(list(years_ticks.items()), list(years_totals.values())):
    print('{}: {}/{} ticks  {} proportion'.format(year, tick, total, tick/total if not total==0 else 'N/A'))

Oct: 0/0 ticks  N/A proportion
Nov: 94/358 ticks  0.26256983240223464 proportion
Dec: 168/555 ticks  0.3027027027027027 proportion
Jan: 165/568 ticks  0.2904929577464789 proportion
Feb: 127/381 ticks  0.3333333333333333 proportion
Mar: 126/431 ticks  0.2923433874709977 proportion
Apr: 59/188 ticks  0.31382978723404253 proportion



2015: 155/548 ticks  0.28284671532846717 proportion
2016: 194/548 ticks  0.354014598540146 proportion
2017: 152/530 ticks  0.28679245283018867 proportion
2018: 153/528 ticks  0.2897727272727273 proportion
2019: 85/327 ticks  0.2599388379204893 proportion


In [98]:
# Tally per season for each team the incorrects
ticks = OrderedDict()
totals = OrderedDict()

# init the dicts of dicts
teams = X_train_pre_drop.Visitor.unique().tolist()
years = [2015, 2016, 2017, 2018, 2019]
for year in years:
    ticks[year] = OrderedDict()
    totals[year] = OrderedDict()
    for team in teams:
        ticks[year][team] = 0
        totals[year][team] = 0

# tally
for (r, row), inc in zip(X_test_pre_drop.iterrows(), incorrects):
    date_regex = re.match(r'\w+\s(\w+)\s(\d+)\s(\d+)', row['Date'])
    month = date_regex.group(1)
    year = date_regex.group(3)
    which_seas = which_season(year, month)
    v = row['Visitor']
    h = row['Home']
    totals[which_seas][v] += 1
    totals[which_seas][h] += 1
    if not inc:
        continue
    ticks[which_seas][v] += 1
    ticks[which_seas][h] += 1

# print
for year, teams_dict in list(ticks.items()):
    print(' --- {} ---'.format(year))
    for (team, tick), total in zip(list(teams_dict.items()), list(totals[year].values())):
        print('{}: {}/{} ticks  {} proportion'.format(team, tick, total, tick/total))
    print('\n')

 --- 2015 ---
Orlando Magic: 13/46 ticks  0.2826086956521739 proportion
San Antonio Spurs: 4/31 ticks  0.12903225806451613 proportion
Brooklyn Nets: 9/33 ticks  0.2727272727272727 proportion
Philadelphia 76ers: 6/42 ticks  0.14285714285714285 proportion
Memphis Grizzlies: 12/38 ticks  0.3157894736842105 proportion
Detroit Pistons: 12/34 ticks  0.35294117647058826 proportion
Cleveland Cavaliers: 9/38 ticks  0.23684210526315788 proportion
Indiana Pacers: 10/32 ticks  0.3125 proportion
Golden State Warriors: 4/31 ticks  0.12903225806451613 proportion
Dallas Mavericks: 12/36 ticks  0.3333333333333333 proportion
Denver Nuggets: 14/32 ticks  0.4375 proportion
Portland Trail Blazers: 11/33 ticks  0.3333333333333333 proportion
New York Knicks: 7/30 ticks  0.23333333333333334 proportion
Washington Wizards: 6/39 ticks  0.15384615384615385 proportion
Sacramento Kings: 15/36 ticks  0.4166666666666667 proportion
Atlanta Hawks: 8/31 ticks  0.25806451612903225 proportion
Charlotte Hornets: 7/31 ticks

In [99]:
# BASELINE

# Tally per season for each team the incorrects
ticks = OrderedDict()
totals = OrderedDict()

# init the dicts of dicts
teams = X_train_pre_drop.Visitor.unique().tolist()
years = [2015, 2016, 2017, 2018, 2019]
for year in years:
    ticks[year] = OrderedDict()
    totals[year] = OrderedDict()
    for team in teams:
        ticks[year][team] = 0
        totals[year][team] = 0

# tally
for (r, row), inc in zip(X_test_pre_drop.iterrows(), incorrects_baseline):
    date_regex = re.match(r'\w+\s(\w+)\s(\d+)\s(\d+)', row['Date'])
    month = date_regex.group(1)
    year = date_regex.group(3)
    which_seas = which_season(year, month)
    v = row['Visitor']
    h = row['Home']
    totals[which_seas][v] += 1
    totals[which_seas][h] += 1
    if not inc:
        continue
    ticks[which_seas][v] += 1
    ticks[which_seas][h] += 1

# print
for year, teams_dict in list(ticks.items()):
    print(' --- {} ---'.format(year))
    for (team, tick), total in zip(list(teams_dict.items()), list(totals[year].values())):
        print('{}: {}/{} ticks  {} proportion'.format(team, tick, total, tick/total))
    print('\n')

 --- 2015 ---
Orlando Magic: 14/46 ticks  0.30434782608695654 proportion
San Antonio Spurs: 5/31 ticks  0.16129032258064516 proportion
Brooklyn Nets: 8/33 ticks  0.24242424242424243 proportion
Philadelphia 76ers: 7/42 ticks  0.16666666666666666 proportion
Memphis Grizzlies: 9/38 ticks  0.23684210526315788 proportion
Detroit Pistons: 12/34 ticks  0.35294117647058826 proportion
Cleveland Cavaliers: 9/38 ticks  0.23684210526315788 proportion
Indiana Pacers: 9/32 ticks  0.28125 proportion
Golden State Warriors: 4/31 ticks  0.12903225806451613 proportion
Dallas Mavericks: 11/36 ticks  0.3055555555555556 proportion
Denver Nuggets: 10/32 ticks  0.3125 proportion
Portland Trail Blazers: 11/33 ticks  0.3333333333333333 proportion
New York Knicks: 8/30 ticks  0.26666666666666666 proportion
Washington Wizards: 14/39 ticks  0.358974358974359 proportion
Sacramento Kings: 15/36 ticks  0.4166666666666667 proportion
Atlanta Hawks: 7/31 ticks  0.22580645161290322 proportion
Charlotte Hornets: 8/31 tick

In [100]:
# Look at a team for a season
team_chosen = 'Miami Heat'
year_chosen = 2018
df_team = pd.DataFrame(columns=X_test_pre_drop.columns.tolist())
for (r, row), inc in zip(X_test_pre_drop.iterrows(), incorrects):
    date_regex = re.match(r'\w+\s(\w+)\s(\d+)\s(\d+)', row['Date'])
    month = date_regex.group(1)
    year = date_regex.group(3)
    which_seas = which_season(year, month)
    if which_seas == year_chosen and inc and (row['Visitor'] == team_chosen or row['Home'] == team_chosen):
        df_team = df_team.append(row, ignore_index=False)
df_team

,Date,Visitor,PTS-V,Home,PTS-H,W-V,L-V,W-H,L-H,Rest-V,...,PIE-H,PIE-V,REB%-H,REB%-V,TOV%-H,TOV%-V,TS%-H,TS%-V,eFG%-H,eFG%-V
46,Thu Dec 20 2018,Houston Rockets,99,Miami Heat,101,16,15,14,16,1,...,50.1,49.8,50.9,48.1,14.9,13.5,54.2,58.1,51.5,54.2
516,Mon Feb 25 2019,Phoenix Suns,124,Miami Heat,121,12,50,26,33,2,...,50.1,45.0,50.9,46.4,14.9,15.3,54.2,55.2,51.5,51.4
805,Sat Nov 10 2018,Washington Wizards,116,Miami Heat,110,3,9,5,7,1,...,50.1,48.7,50.9,47.7,14.9,13.6,54.2,56.7,51.5,53.1
1030,Sun Jan 6 2019,Miami Heat,82,Atlanta Hawks,106,19,19,12,27,2,...,46.4,50.1,49.6,50.9,16.2,14.9,55.5,54.2,52.2,51.5
1473,Wed Apr 10 2019,Miami Heat,94,Brooklyn Nets,113,39,43,42,40,1,...,49.2,50.1,50.1,50.9,14.6,14.9,55.6,54.2,52.0,51.5
1508,Wed Nov 14 2018,Miami Heat,120,Brooklyn Nets,107,6,8,6,9,2,...,49.2,50.1,50.1,50.9,14.6,14.9,55.6,54.2,52.0,51.5
1541,Tue Apr 9 2019,Philadelphia 76ers,99,Miami Heat,122,50,31,39,42,2,...,50.1,52.9,50.9,51.7,14.9,14.4,54.2,57.4,51.5,53.2
1614,Thu Jan 10 2019,Boston Celtics,99,Miami Heat,115,25,16,20,20,1,...,50.1,53.0,50.9,49.5,14.9,12.7,54.2,56.7,51.5,53.4
1747,Sat Dec 22 2018,Milwaukee Bucks,87,Miami Heat,94,22,10,15,16,1,...,50.1,55.3,50.9,51.6,14.9,13.3,54.2,58.3,51.5,55.0
1833,Wed Jan 30 2019,Chicago Bulls,105,Miami Heat,89,12,40,24,25,1,...,50.1,45.3,50.9,48.3,14.9,14.1,54.2,54.1,51.5,50.5


In [101]:
# BASELINE

# Look at a team for a season
team_chosen = 'Miami Heat'
year_chosen = 2018
df_team = pd.DataFrame(columns=X_test_pre_drop.columns.tolist())
for (r, row), inc in zip(X_test_pre_drop.iterrows(), incorrects_baseline):
    date_regex = re.match(r'\w+\s(\w+)\s(\d+)\s(\d+)', row['Date'])
    month = date_regex.group(1)
    year = date_regex.group(3)
    which_seas = which_season(year, month)
    if which_seas == year_chosen and inc and (row['Visitor'] == team_chosen or row['Home'] == team_chosen):
        df_team = df_team.append(row, ignore_index=False)
df_team

,Date,Visitor,PTS-V,Home,PTS-H,W-V,L-V,W-H,L-H,Rest-V,...,PIE-H,PIE-V,REB%-H,REB%-V,TOV%-H,TOV%-V,TS%-H,TS%-V,eFG%-H,eFG%-V
46,Thu Dec 20 2018,Houston Rockets,99,Miami Heat,101,16,15,14,16,1,...,50.1,49.8,50.9,48.1,14.9,13.5,54.2,58.1,51.5,54.2
516,Mon Feb 25 2019,Phoenix Suns,124,Miami Heat,121,12,50,26,33,2,...,50.1,45.0,50.9,46.4,14.9,15.3,54.2,55.2,51.5,51.4
657,Sat Mar 2 2019,Brooklyn Nets,88,Miami Heat,117,32,33,28,34,1,...,50.1,49.2,50.9,50.1,14.9,14.6,54.2,55.6,51.5,52.0
805,Sat Nov 10 2018,Washington Wizards,116,Miami Heat,110,3,9,5,7,1,...,50.1,48.7,50.9,47.7,14.9,13.6,54.2,56.7,51.5,53.1
1030,Sun Jan 6 2019,Miami Heat,82,Atlanta Hawks,106,19,19,12,27,2,...,46.4,50.1,49.6,50.9,16.2,14.9,55.5,54.2,52.2,51.5
1508,Wed Nov 14 2018,Miami Heat,120,Brooklyn Nets,107,6,8,6,9,2,...,49.2,50.1,50.1,50.9,14.6,14.9,55.6,54.2,52.0,51.5
1541,Tue Apr 9 2019,Philadelphia 76ers,99,Miami Heat,122,50,31,39,42,2,...,50.1,52.9,50.9,51.7,14.9,14.4,54.2,57.4,51.5,53.2
1614,Thu Jan 10 2019,Boston Celtics,99,Miami Heat,115,25,16,20,20,1,...,50.1,53.0,50.9,49.5,14.9,12.7,54.2,56.7,51.5,53.4
1747,Sat Dec 22 2018,Milwaukee Bucks,87,Miami Heat,94,22,10,15,16,1,...,50.1,55.3,50.9,51.6,14.9,13.3,54.2,58.3,51.5,55.0
1833,Wed Jan 30 2019,Chicago Bulls,105,Miami Heat,89,12,40,24,25,1,...,50.1,45.3,50.9,48.3,14.9,14.1,54.2,54.1,51.5,50.5


In [102]:
# Print
index = 0
for t, p, b in zip(y_test.values.flatten(), y_pred, baseline_preds):
    add_on = 'WRONG' if t != p else ''
    print('{}: real {}  pred {}  baseline: {}  {}'.format(index, t, p, b, add_on))
    index += 1

0: real 0  pred 0  baseline: 0  
1: real 1  pred 1  baseline: 1  
2: real 1  pred 0  baseline: 0  WRONG
3: real 0  pred 1  baseline: 1  WRONG
4: real 0  pred 0  baseline: 0  
5: real 0  pred 1  baseline: 1  WRONG
6: real 1  pred 1  baseline: 1  
7: real 0  pred 0  baseline: 0  
8: real 0  pred 1  baseline: 1  WRONG
9: real 1  pred 0  baseline: 0  WRONG
10: real 0  pred 0  baseline: 0  
11: real 0  pred 1  baseline: 0  WRONG
12: real 1  pred 0  baseline: 0  WRONG
13: real 1  pred 1  baseline: 1  
14: real 0  pred 1  baseline: 0  WRONG
15: real 1  pred 1  baseline: 1  
16: real 1  pred 0  baseline: 0  WRONG
17: real 0  pred 0  baseline: 0  
18: real 1  pred 1  baseline: 1  
19: real 1  pred 1  baseline: 1  
20: real 1  pred 1  baseline: 1  
21: real 1  pred 1  baseline: 1  
22: real 1  pred 1  baseline: 1  
23: real 1  pred 1  baseline: 1  
24: real 0  pred 0  baseline: 0  
25: real 0  pred 0  baseline: 0  
26: real 0  pred 0  baseline: 0  
27: real 0  pred 1  baseline: 1  WRONG
28: real

1932: real 1  pred 1  baseline: 1  
1933: real 1  pred 1  baseline: 1  
1934: real 1  pred 0  baseline: 0  WRONG
1935: real 1  pred 1  baseline: 1  
1936: real 1  pred 1  baseline: 1  
1937: real 1  pred 1  baseline: 1  
1938: real 1  pred 1  baseline: 1  
1939: real 1  pred 1  baseline: 1  
1940: real 1  pred 1  baseline: 1  
1941: real 1  pred 1  baseline: 1  
1942: real 0  pred 1  baseline: 0  WRONG
1943: real 1  pred 1  baseline: 1  
1944: real 1  pred 0  baseline: 0  WRONG
1945: real 1  pred 1  baseline: 1  
1946: real 1  pred 1  baseline: 1  
1947: real 1  pred 1  baseline: 1  
1948: real 1  pred 1  baseline: 1  
1949: real 1  pred 1  baseline: 1  
1950: real 0  pred 0  baseline: 0  
1951: real 1  pred 1  baseline: 0  
1952: real 1  pred 1  baseline: 0  
1953: real 1  pred 1  baseline: 1  
1954: real 1  pred 1  baseline: 1  
1955: real 0  pred 1  baseline: 0  WRONG
1956: real 1  pred 0  baseline: 0  WRONG
1957: real 1  pred 1  baseline: 1  
1958: real 0  pred 0  baseline: 0  
195

In [103]:
# Probs
y_prob = grid_search_xgb.best_estimator_.predict_proba(X_test_data).tolist()

In [106]:
from statistics import mean, median

# Get avg diff for probs total, for each incorrect pred
probs_diff_total = []  # all diff
probs_diff_cor = []  # When the model was correct
probs_diff_inc = []  # when the model was incorrect
probs_diff_base = []  # when the baseline was incorect
probs_diff_base_cor_model_inc = []  # when the baseline was correct (ie no upset) but the model predicted an upset
probs_diff_base_inc_model_cor = []  # when the model predicted an upset correctly

score_diff_total = []  # all diff
score_diff_cor = []  # When the model was correct
score_diff_inc = []  # when the model was incorrect
score_diff_base = []  # when the baseline was incorect
score_diff_base_cor_model_inc = []  # when the baseline was correct (ie no upset) but the model predicted an upset
score_diff_base_inc_model_cor = []  # when the model predicted an upset correctly

for t, p, pr, b, (_, row) in zip(y_test.values.flatten(), y_pred, y_prob, baseline_preds, X_test_pre_drop.iterrows()):
    diff = abs(pr[0] - pr[1])
    diff_pts = abs(row['PTS-V'] - row['PTS-H'])
    probs_diff_total.append(diff)
    score_diff_total.append(diff_pts)
    if t != p:
        probs_diff_inc.append(diff)
        score_diff_inc.append(diff_pts)
    if t == p:
        probs_diff_cor.append(diff)
        score_diff_cor.append(diff_pts)
    if b != t:
        probs_diff_base.append(diff)
        score_diff_base.append(diff_pts)
    if b == t and t != p:
        probs_diff_base_cor_model_inc.append(diff)
        score_diff_base_cor_model_inc.append(diff_pts)
    if b != t and t == p:
        probs_diff_base_inc_model_cor.append(diff)
        score_diff_base_inc_model_cor.append(diff_pts)

print('Probs diff avg total: Probs - {:.3f}  {:.3f}   Score -  {:.3f}  {:.3f}'.format(mean(probs_diff_total), median(probs_diff_total), mean(score_diff_total), median(score_diff_total)))
print('Probs diff avg cor: Probs - {:.3f}  {:.3f}   Score -  {:.3f}  {:.3f}'.format(mean(probs_diff_cor), median(probs_diff_cor), mean(score_diff_cor), median(score_diff_cor)))
print('Probs diff avg inc: Probs - {:.3f}  {:.3f}   Score -  {:.3f}  {:.3f}'.format(mean(probs_diff_inc), median(probs_diff_inc), mean(score_diff_inc), median(score_diff_inc)))
print('Probs diff avg base inc: Probs - {:.3f}  {:.3f}   Score -  {:.3f}  {:.3f}'.format(mean(probs_diff_base), median(probs_diff_base), mean(score_diff_base), median(score_diff_base)))
print('Probs diff avg base cor model inc: Probs - {:.3f}  {:.3f}   Score -  {:.3f}  {:.3f}'.format(mean(probs_diff_base_cor_model_inc), median(probs_diff_base_cor_model_inc), mean(score_diff_base_cor_model_inc), median(score_diff_base_cor_model_inc)))
print('Probs diff avg base inc model cor: Probs - {:.3f}  {:.3f}   Score -  {:.3f}  {:.3f}'.format(mean(probs_diff_base_inc_model_cor), median(probs_diff_base_inc_model_cor), mean(score_diff_base_inc_model_cor), median(score_diff_base_inc_model_cor)))

Probs diff avg total: Probs - 0.298  0.275   Score -  11.308  9.000
Probs diff avg cor: Probs - 0.327  0.316   Score -  12.326  10.000
Probs diff avg inc: Probs - 0.231  0.196   Score -  8.967  7.000
Probs diff avg base inc: Probs - 0.235  0.214   Score -  9.936  8.000
Probs diff avg base cor model inc: Probs - 0.154  0.134   Score -  8.815  8.000
Probs diff avg base inc model cor: Probs - 0.162  0.139   Score -  12.078  10.000


In [105]:
# Remove preds that corrospond to prob diff outside of some margin
margin = 0.1
new_preds = []
new_test = []
for t, p, pr in zip(y_test.values.flatten(), y_pred, y_prob):
    if abs(pr[0] - pr[1]) > margin:
        new_preds.append(p)
        new_test.append(t)
print('{:.3f}% of orginal data included'.format(len(new_preds) / len(y_pred)))
accuracy_score(new_preds, new_test)

0.819% of orginal data included


0.723781388478582